In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, cohen_kappa_score
from scipy.signal import resample
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Load datasets
eeg_data = pd.read_csv('eeg_data_trimmed.csv')
emg_data = pd.read_csv('emg_data_trimmed.csv')
labels = pd.read_csv('labels_trimmed.csv')

# Label encoding
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels.values.ravel())

# Ensure combined_data and labels are aligned
if len(labels) > len(eeg_data):
    labels = labels[:len(eeg_data)]
elif len(labels) < len(eeg_data):
    eeg_data = eeg_data.iloc[:len(labels)]
    emg_data = emg_data.iloc[:len(labels)]

# Resample EEG to match EMG's length (if necessary)
if len(eeg_data) != len(emg_data):
    eeg_data = resample(eeg_data, num=len(emg_data))

# Dynamic Time Warping (DTW) for offline alignment
distance, path = fastdtw(eeg_data.values, emg_data.values, dist=euclidean)
aligned_eeg = eeg_data.iloc[[p[0] for p in path]]
aligned_emg = emg_data.iloc[[p[1] for p in path]]

# Combine aligned data
combined_data = pd.concat([aligned_eeg.reset_index(drop=True), aligned_emg.reset_index(drop=True)], axis=1)

# Ensure combined_data and labels are still aligned
if len(labels) > len(combined_data):
    labels = labels[:len(combined_data)]
elif len(labels) < len(combined_data):
    combined_data = combined_data.iloc[:len(labels)]

class EEGEMGDataset(Dataset):
    def __init__(self, X, y, noise_std=0.01):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
        self.noise_std = noise_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X_noisy = self.X[idx] + torch.normal(0, self.noise_std, size=self.X[idx].shape)
        return X_noisy, self.y[idx]

# CNN-GRU Model
class CNNGRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, kernel_size=3, num_filters=16, dropout_rate=0.7):
        super(CNNGRUModel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.gru = nn.GRU(input_size=num_filters, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  
        cnn_out = self.cnn(x).permute(0, 2, 1)  
        gru_out, _ = self.gru(cnn_out)
        output = self.fc(gru_out[:, -1, :])  
        return output

# Train function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=25, patience=5, model_save_path="best_model.pth"):
    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}, Val Loss: {val_loss:.4f}")

        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict()
            torch.save(best_model_state, model_save_path)  
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                model.load_state_dict(best_model_state)
                break



# Online Learning Function
def online_adaptation(model, optimizer, buffer_X, buffer_y, criterion, val_loader, num_cycles=3, batch_size=16, lr=0.0001):
    model.train()
    buffer_dataset = EEGEMGDataset(buffer_X, buffer_y)
    buffer_loader = DataLoader(buffer_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.005)

    for cycle in range(num_cycles):
        print(f"Online Adaptation Cycle {cycle + 1}/{num_cycles}")
        for X_batch, y_batch in buffer_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        print("Validation Metrics After Cycle:")
        evaluate_model(model, val_loader)

from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef

# Compute Metrics Function (Updated)
def compute_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    kappa_score = cohen_kappa_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    return precision, recall, f1, accuracy, balanced_acc, kappa_score, mcc

# Evaluation function (Updated)
def evaluate_model(model, val_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    precision, recall, f1, accuracy, balanced_acc, kappa_score, mcc = compute_metrics(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}, Accuracy: {accuracy:.2f}")
    print(f"Balanced Accuracy: {balanced_acc:.2f}, Cohen's Kappa: {kappa_score:.2f}, MCC: {mcc:.2f}")
    print("Confusion Matrix:")
    print(conf_matrix)

    return precision, recall, f1, accuracy, balanced_acc, kappa_score, mcc, conf_matrix
# K-Fold Cross-Validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
fold_results = []
online_adaptation_percentage = 0.3  
# Updated evaluation after training and online adaptation
for fold, (train_index, val_index) in enumerate(kf.split(combined_data)):
    print(f"Fold {fold + 1}/{k}")
    X_train, X_val = combined_data.iloc[train_index].values, combined_data.iloc[val_index].values
    y_train, y_val = labels[train_index], labels[val_index]

    adaptation_size = int(len(X_val) * online_adaptation_percentage)
    buffer_X, buffer_y = X_val[:adaptation_size], y_val[:adaptation_size]
    X_val, y_val = X_val[adaptation_size:], y_val[adaptation_size:]

    train_dataset = EEGEMGDataset(X_train, y_train, noise_std=0.02)
    val_dataset = EEGEMGDataset(X_val, y_val, noise_std=0.0)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model = CNNGRUModel(input_dim=X_train.shape[1], hidden_dim=128, num_classes=len(np.unique(labels)), num_filters=16, dropout_rate=0.7)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

    train_model(model, train_loader, val_loader, criterion, optimizer)
    
    print("\nInitial Evaluation Before Online Learning:")
    evaluate_model(model, val_loader)
    
    online_adaptation(model, optimizer, buffer_X, buffer_y, criterion, val_loader)
    
    print("\nFinal Evaluation After Online Learning:")
    evaluate_model(model, val_loader)


Fold 1/5


/home/tipu/.local/lib/python3.10/site-packages/torch/autograd/graph.py:769: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/25, Loss: 1.5923, Val Loss: 1.3230
Epoch 2/25, Loss: 1.1124, Val Loss: 0.8826
Epoch 3/25, Loss: 0.9497, Val Loss: 0.8220
Epoch 4/25, Loss: 0.9899, Val Loss: 1.1143
Epoch 5/25, Loss: 1.0224, Val Loss: 0.8193
Epoch 6/25, Loss: 0.8827, Val Loss: 0.7757
Epoch 7/25, Loss: 0.8894, Val Loss: 0.8192
Epoch 8/25, Loss: 0.8823, Val Loss: 0.7244
Epoch 9/25, Loss: 0.7211, Val Loss: 0.6994
Epoch 10/25, Loss: 2.2459, Val Loss: 0.9549
Epoch 11/25, Loss: 0.8090, Val Loss: 0.6678
Epoch 12/25, Loss: 0.6433, Val Loss: 0.6018
Epoch 13/25, Loss: 0.6588, Val Loss: 0.6652
Epoch 14/25, Loss: 0.5940, Val Loss: 0.6675
Epoch 15/25, Loss: 0.5584, Val Loss: 0.5769
Epoch 16/25, Loss: 0.5275, Val Loss: 0.5645
Epoch 17/25, Loss: 0.4365, Val Loss: 0.4196
Epoch 18/25, Loss: 0.3946, Val Loss: 0.4303
Epoch 19/25, Loss: 1.2368, Val Loss: 0.6968
Epoch 20/25, Loss: 0.8336, Val Loss: 0.6154
Epoch 21/25, Loss: 0.5162, Val Loss: 0.5144
Epoch 22/25, Loss: 0.4011, Val Loss: 0.5572
Early stopping triggered.

Initial Evalua

/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.79, Recall: 0.74, F1 Score: 0.70, Accuracy: 0.74
Balanced Accuracy: 0.62, Cohen's Kappa: 0.67, MCC: 0.70
Confusion Matrix:
[[  0   4   0   0   0   0]
 [  0 184   0   0   0   0]
 [  0   0 174  16   0   0]
 [  0   0  46 115   7  11]
 [  0   0   0   0  26 158]
 [  0   0   0   0   0 186]]
Online Adaptation Cycle 1/3
Validation Metrics After Cycle:


...

/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Precision: 0.35, Recall: 0.23, F1 Score: 0.23, Accuracy: 0.23
Balanced Accuracy: 0.22, Cohen's Kappa: 0.12, MCC: 0.13
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0 144   0   0   0   0   0]
 [ 61  65   0  13  37   5  33]
 [ 39  15   0   8  21  65  30]
 [ 19   0   0   1   9  83  83]
 [  6   0   0   0   1  92  97]]
Online Adaptation Cycle 3/3
Validation Metrics After Cycle:


/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Precision: 0.37, Recall: 0.22, F1 Score: 0.24, Accuracy: 0.22
Balanced Accuracy: 0.21, Cohen's Kappa: 0.10, MCC: 0.11
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0 144   0   0   0   0   0]
 [ 55  40   0  27  30   6  56]
 [ 44  13   0  15  30  55  21]
 [ 27   0   0   0   1  83  84]
 [ 30   0   0   0   0 105  61]]

Final Evaluation After Online Learning:
Precision: 0.37, Recall: 0.22, F1 Score: 0.24, Accuracy: 0.22
Balanced Accuracy: 0.21, Cohen's Kappa: 0.10, MCC: 0.11
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0 144   0   0   0   0   0]
 [ 55  40   0  27  30   6  56]
 [ 44  13   0  15  30  55  21]
 [ 27   0   0   0   1  83  84]
 [ 30   0   0   0   0 105  61]]


/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [4]:
import numpy as np

# CNNGRU metrics
cnn_gru_metrics = [
    [0.84, 0.78, 0.76, 0.78, 0.82, 0.72, 0.74],
    [0.79, 0.74, 0.70, 0.74, 0.62, 0.67, 0.70],
    [0.57, 0.55, 0.54, 0.55, 0.54, 0.46, 0.49],
    [0.10, 0.20, 0.12, 0.20, 0.21, 0.05, 0.07],
    [0.39, 0.31, 0.29, 0.31, 0.29, 0.17, 0.19]
]

# Online metrics
online_metrics = [
    [0.85, 0.81, 0.81, 0.81, 0.85, 0.77, 0.78],
    [0.61, 0.57, 0.55, 0.57, 0.52, 0.49, 0.53],
    [0.58, 0.50, 0.45, 0.50, 0.47, 0.39, 0.42],
    [0.07, 0.04, 0.05, 0.04, 0.04, 0.02, 0.04],
    [0.34, 0.24, 0.25, 0.24, 0.23, 0.11, 0.12]
]

# Calculate averages
cnn_gru_avg = np.mean(cnn_gru_metrics, axis=0)
online_avg = np.mean(online_metrics, axis=0)

# Metric labels
metric_labels = [
    "Precision", "Recall", "F1 Score", "Accuracy", "Balanced Accuracy", "Cohen's Kappa", "MCC"
]

# Display results
def display_results(label, averages):
    print(f"Average metrics for {label}:")
    for metric, avg in zip(metric_labels, averages):
        print(f"{metric}: {avg:.2f}")
    print()

display_results("CNNGRU", cnn_gru_avg)
display_results("Online", online_avg)


Average metrics for CNNGRU:
Precision: 0.54
Recall: 0.52
F1 Score: 0.48
Accuracy: 0.52
Balanced Accuracy: 0.50
Cohen's Kappa: 0.41
MCC: 0.44

Average metrics for Online:
Precision: 0.49
Recall: 0.43
F1 Score: 0.42
Accuracy: 0.43
Balanced Accuracy: 0.42
Cohen's Kappa: 0.36
MCC: 0.38



In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, cohen_kappa_score
from scipy.signal import resample
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Load datasets
eeg_data = pd.read_csv('eeg_data_trimmed.csv')
emg_data = pd.read_csv('emg_data_trimmed.csv')
labels = pd.read_csv('labels_trimmed.csv')

# Label encoding
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels.values.ravel())

# Ensure combined_data and labels are aligned
if len(labels) > len(eeg_data):
    labels = labels[:len(eeg_data)]
elif len(labels) < len(eeg_data):
    eeg_data = eeg_data.iloc[:len(labels)]
    emg_data = emg_data.iloc[:len(labels)]

# Resample EEG to match EMG's length (if necessary)
if len(eeg_data) != len(emg_data):
    eeg_data = resample(eeg_data, num=len(emg_data))

# Dynamic Time Warping (DTW) for offline alignment
distance, path = fastdtw(eeg_data.values, emg_data.values, dist=euclidean)
aligned_eeg = eeg_data.iloc[[p[0] for p in path]]
aligned_emg = emg_data.iloc[[p[1] for p in path]]

# Combine aligned data
combined_data = pd.concat([aligned_eeg.reset_index(drop=True), aligned_emg.reset_index(drop=True)], axis=1)

# Ensure combined_data and labels are still aligned
if len(labels) > len(combined_data):
    labels = labels[:len(combined_data)]
elif len(labels) < len(combined_data):
    combined_data = combined_data.iloc[:len(labels)]

class EEGEMGDataset(Dataset):
    def __init__(self, X, y, noise_std=0.002):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
        self.noise_std = noise_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X_noisy = self.X[idx] + torch.normal(0, self.noise_std, size=self.X[idx].shape)
        return X_noisy, self.y[idx]

# CNN-RNN Model
class CNNRNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_filters=16, dropout_rate=0.7):
        super(CNNRNNModel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.rnn = nn.RNN(input_size=num_filters, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        x = x.unsqueeze(1)
        cnn_out = self.cnn(x).permute(0, 2, 1)
        rnn_out, _ = self.rnn(cnn_out)
        output = self.fc(rnn_out[:, -1, :])
        return output

# TCN Model
class TCNModel(nn.Module):
    def __init__(self, input_dim, num_classes, num_filters=16, dropout_rate=0.7):
        super(TCNModel, self).__init__()
        self.tcn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=num_filters, out_channels=num_filters, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate)
        )
        self.fc = nn.Linear(num_filters, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        tcn_out = self.tcn(x).mean(dim=2)
        output = self.fc(tcn_out)
        return output

# Online Adaptation Function
def online_adaptation(model, optimizer, buffer_X, buffer_y, criterion, val_loader, num_cycles=3, batch_size=16, lr=0.0001):
    model.train()
    buffer_dataset = EEGEMGDataset(buffer_X, buffer_y)
    buffer_loader = DataLoader(buffer_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.005)

    for cycle in range(num_cycles):
        print(f"Online Adaptation Cycle {cycle + 1}/{num_cycles}")
        for X_batch, y_batch in buffer_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        print("Validation Metrics After Cycle:")
        evaluate_model(model, val_loader)

from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef

# Compute Metrics Function (Updated)
def compute_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    kappa_score = cohen_kappa_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    return precision, recall, f1, accuracy, balanced_acc, kappa_score, mcc

# Evaluation function (Updated)
def evaluate_model(model, val_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    precision, recall, f1, accuracy, balanced_acc, kappa_score, mcc = compute_metrics(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}, Accuracy: {accuracy:.2f}")
    print(f"Balanced Accuracy: {balanced_acc:.2f}, Cohen's Kappa: {kappa_score:.2f}, MCC: {mcc:.2f}")
    print("Confusion Matrix:")
    print(conf_matrix)

    return precision, recall, f1, accuracy, balanced_acc, kappa_score, mcc, conf_matrix
# K-Fold Cross-Validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
fold_results = []
online_adaptation_percentage = 0.3  
# Updated evaluation after training and online adaptation
for fold, (train_index, val_index) in enumerate(kf.split(combined_data)):
    print(f"Fold {fold + 1}/{k}")
    X_train, X_val = combined_data.iloc[train_index].values, combined_data.iloc[val_index].values
    y_train, y_val = labels[train_index], labels[val_index]

    adaptation_size = int(len(X_val) * online_adaptation_percentage)
    buffer_X, buffer_y = X_val[:adaptation_size], y_val[:adaptation_size]
    X_val, y_val = X_val[adaptation_size:], y_val[adaptation_size:]

    train_dataset = EEGEMGDataset(X_train, y_train, noise_std=0.02)
    val_dataset = EEGEMGDataset(X_val, y_val, noise_std=0.0)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model = CNNRNNModel(input_dim=X_train.shape[1], hidden_dim=128, num_classes=len(np.unique(labels)), num_filters=16, dropout_rate=0.7)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

    train_model(model, train_loader, val_loader, criterion, optimizer)
    
    print("\nInitial Evaluation Before Online Learning:")
    evaluate_model(model, val_loader)
    
    online_adaptation(model, optimizer, buffer_X, buffer_y, criterion, val_loader)
    
    print("\nFinal Evaluation After Online Learning:")
    evaluate_model(model, val_loader)


Fold 1/5
Epoch 1/25, Loss: 1.6195, Val Loss: 1.3442
Epoch 2/25, Loss: 1.2834, Val Loss: 1.2130
Epoch 3/25, Loss: 1.1688, Val Loss: 0.9861
Epoch 4/25, Loss: 1.0323, Val Loss: 0.8917
Epoch 5/25, Loss: 0.9839, Val Loss: 0.8372
Epoch 6/25, Loss: 0.9208, Val Loss: 0.8595
Epoch 7/25, Loss: 0.8899, Val Loss: 0.8812
Epoch 8/25, Loss: 0.8805, Val Loss: 0.7912
Epoch 9/25, Loss: 1.0152, Val Loss: 0.8359
Epoch 10/25, Loss: 0.9482, Val Loss: 0.8839
Epoch 11/25, Loss: 0.9264, Val Loss: 0.8194
Epoch 12/25, Loss: 0.8846, Val Loss: 0.8164
Epoch 13/25, Loss: 0.8304, Val Loss: 0.7534
Epoch 14/25, Loss: 0.7733, Val Loss: 0.6924
Epoch 15/25, Loss: 0.7267, Val Loss: 0.7917
Epoch 16/25, Loss: 0.6429, Val Loss: 0.7398
Epoch 17/25, Loss: 0.6169, Val Loss: 0.7228
Epoch 18/25, Loss: 0.6797, Val Loss: 1.9057
Epoch 19/25, Loss: 1.4635, Val Loss: 1.0729
Early stopping triggered.

Initial Evaluation Before Online Learning:


/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Precision: 0.47, Recall: 0.47, F1 Score: 0.46, Accuracy: 0.47
Balanced Accuracy: 0.55, Cohen's Kappa: 0.37, MCC: 0.39
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   7   0   0   0   0   0]
 [  0 176   0   0   0   0   0]
 [  5   0   0 134  50   2   0]
 [  0   0   0  24  68  80  18]
 [  0   0   0   0  14  82  92]
 [  0   0   0   0   0  30 145]]
Online Adaptation Cycle 1/3
Validation Metrics After Cycle:


/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Precision: 0.34, Recall: 0.44, F1 Score: 0.38, Accuracy: 0.44
Balanced Accuracy: 0.36, Cohen's Kappa: 0.33, MCC: 0.36
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  7   0   0   0   0   0   0]
 [176   0   0   0   0   0   0]
 [  0   0   0 186   5   0   0]
 [  0   0   0  73  78  39   0]
 [  0   0   0   0  43 145   0]
 [  0   0   0   0  30 145   0]]
Online Adaptation Cycle 2/3
Validation Metrics After Cycle:


...

/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Precision: 0.04, Recall: 0.10, F1 Score: 0.05, Accuracy: 0.10
Balanced Accuracy: 0.11, Cohen's Kappa: -0.00, MCC: -0.01
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [ 27  71   0   0  46   0   0]
 [  0 112   0   0 102   0   0]
 [  0  81   0   0  97   0   0]
 [  0  76   0   0 119   0   0]
 [  0  35   0   0 161   0   0]]
Online Adaptation Cycle 3/3
Validation Metrics After Cycle:


/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Precision: 0.04, Recall: 0.04, F1 Score: 0.04, Accuracy: 0.04
Balanced Accuracy: 0.04, Cohen's Kappa: -0.00, MCC: -0.00
Confusion Matrix:
[[  0   0   0   0   0   0]
 [117   0   0  27   0   0]
 [175   0   0  39   0   0]
 [139   0   0  39   0   0]
 [156   0   0  39   0   0]
 [133   0   0  63   0   0]]

Final Evaluation After Online Learning:
Precision: 0.04, Recall: 0.04, F1 Score: 0.04, Accuracy: 0.04
Balanced Accuracy: 0.04, Cohen's Kappa: -0.00, MCC: -0.00
Confusion Matrix:
[[  0   0   0   0   0   0]
 [117   0   0  27   0   0]
 [175   0   0  39   0   0]
 [139   0   0  39   0   0]
 [156   0   0  39   0   0]
 [133   0   0  63   0   0]]


/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [5]:
import numpy as np

# CNNGRU metrics
cnn_gru_metrics = [
    [0.84, 0.78, 0.76, 0.78, 0.82, 0.72, 0.74],
    [0.79, 0.74, 0.70, 0.74, 0.62, 0.67, 0.70],
    [0.57, 0.55, 0.54, 0.55, 0.54, 0.46, 0.49],
    [0.10, 0.20, 0.12, 0.20, 0.21, 0.05, 0.07],
    [0.39, 0.31, 0.29, 0.31, 0.29, 0.17, 0.19]
]

# Online metrics for CNNGRU
online_metrics_cnn_gru = [
    [0.85, 0.81, 0.81, 0.81, 0.85, 0.77, 0.78],
    [0.61, 0.57, 0.55, 0.57, 0.52, 0.49, 0.53],
    [0.58, 0.50, 0.45, 0.50, 0.47, 0.39, 0.42],
    [0.07, 0.04, 0.05, 0.04, 0.04, 0.02, 0.04],
    [0.34, 0.24, 0.25, 0.24, 0.23, 0.11, 0.12]
]

# CNNRNN metrics
cnn_rnn_metrics = [
    [0.47, 0.47, 0.46, 0.47, 0.55, 0.37, 0.39],
    [0.16, 0.25, 0.18, 0.25, 0.21, 0.08, 0.10],
    [0.36, 0.38, 0.33, 0.38, 0.39, 0.22, 0.26],
    [0.22, 0.28, 0.25, 0.28, 0.29, 0.16, 0.17],
    [0.10, 0.21, 0.10, 0.21, 0.23, 0.03, 0.07]
]

# Online metrics for CNNRNN
online_metrics_cnn_rnn = [
    [0.34, 0.44, 0.38, 0.44, 0.36, 0.33, 0.36],
    [0.16, 0.25, 0.19, 0.25, 0.25, 0.10, 0.11],
    [0.37, 0.22, 0.18, 0.22, 0.23, 0.08, 0.08],
    [0.19, 0.18, 0.11, 0.18, 0.17, 0.05, 0.06],
    [0.13, 0.06, 0.07, 0.06, 0.06, -0.10, -0.12]
]

# Calculate averages
cnn_gru_avg = np.mean(cnn_gru_metrics, axis=0)
online_avg_cnn_gru = np.mean(online_metrics_cnn_gru, axis=0)
cnn_rnn_avg = np.mean(cnn_rnn_metrics, axis=0)
online_avg_cnn_rnn = np.mean(online_metrics_cnn_rnn, axis=0)

# Metric labels
metric_labels = [
    "Precision", "Recall", "F1 Score", "Accuracy", "Balanced Accuracy", "Cohen's Kappa", "MCC"
]

# Display results
def display_results(label, averages):
    print(f"Average metrics for {label}:")
    for metric, avg in zip(metric_labels, averages):
        print(f"{metric}: {avg:.2f}")
    print()

#display_results("CNNGRU", cnn_gru_avg)
#display_results("Online CNNGRU", online_avg_cnn_gru)
display_results("CNNRNN", cnn_rnn_avg)
display_results("Online CNNRNN", online_avg_cnn_rnn)


Average metrics for CNNRNN:
Precision: 0.26
Recall: 0.32
F1 Score: 0.26
Accuracy: 0.32
Balanced Accuracy: 0.33
Cohen's Kappa: 0.17
MCC: 0.20

Average metrics for Online CNNRNN:
Precision: 0.24
Recall: 0.23
F1 Score: 0.19
Accuracy: 0.23
Balanced Accuracy: 0.21
Cohen's Kappa: 0.09
MCC: 0.10



In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, cohen_kappa_score
from scipy.signal import resample
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Load datasets
eeg_data = pd.read_csv('eeg_data_trimmed.csv')
emg_data = pd.read_csv('emg_data_trimmed.csv')
labels = pd.read_csv('labels_trimmed.csv')

# Label encoding
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels.values.ravel())

# Ensure combined_data and labels are aligned
if len(labels) > len(eeg_data):
    labels = labels[:len(eeg_data)]
elif len(labels) < len(eeg_data):
    eeg_data = eeg_data.iloc[:len(labels)]
    emg_data = emg_data.iloc[:len(labels)]

# Resample EEG to match EMG's length (if necessary)
if len(eeg_data) != len(emg_data):
    eeg_data = resample(eeg_data, num=len(emg_data))

# Dynamic Time Warping (DTW) for offline alignment
distance, path = fastdtw(eeg_data.values, emg_data.values, dist=euclidean)
aligned_eeg = eeg_data.iloc[[p[0] for p in path]]
aligned_emg = emg_data.iloc[[p[1] for p in path]]

# Combine aligned data
combined_data = pd.concat([aligned_eeg.reset_index(drop=True), aligned_emg.reset_index(drop=True)], axis=1)

# Ensure combined_data and labels are still aligned
if len(labels) > len(combined_data):
    labels = labels[:len(combined_data)]
elif len(labels) < len(combined_data):
    combined_data = combined_data.iloc[:len(labels)]

class EEGEMGDataset(Dataset):
    def __init__(self, X, y, noise_std=0.002):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
        self.noise_std = noise_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X_noisy = self.X[idx] + torch.normal(0, self.noise_std, size=self.X[idx].shape)
        return X_noisy, self.y[idx]

# CNN-RNN Model
class CNNRNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_filters=16, dropout_rate=0.7):
        super(CNNRNNModel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.rnn = nn.RNN(input_size=num_filters, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        x = x.unsqueeze(1)
        cnn_out = self.cnn(x).permute(0, 2, 1)
        rnn_out, _ = self.rnn(cnn_out)
        output = self.fc(rnn_out[:, -1, :])
        return output

# TCN Model
class TCNModel(nn.Module):
    def __init__(self, input_dim, num_classes, num_filters=16, dropout_rate=0.7):
        super(TCNModel, self).__init__()
        self.tcn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=num_filters, out_channels=num_filters, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate)
        )
        self.fc = nn.Linear(num_filters, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        tcn_out = self.tcn(x).mean(dim=2)
        output = self.fc(tcn_out)
        return output

# Online Adaptation Function
def online_adaptation(model, optimizer, buffer_X, buffer_y, criterion, val_loader, num_cycles=3, batch_size=16, lr=0.0001):
    model.train()
    buffer_dataset = EEGEMGDataset(buffer_X, buffer_y)
    buffer_loader = DataLoader(buffer_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.005)

    for cycle in range(num_cycles):
        print(f"Online Adaptation Cycle {cycle + 1}/{num_cycles}")
        for X_batch, y_batch in buffer_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        print("Validation Metrics After Cycle:")
        evaluate_model(model, val_loader)

from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef

# Compute Metrics Function (Updated)
def compute_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    kappa_score = cohen_kappa_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    return precision, recall, f1, accuracy, balanced_acc, kappa_score, mcc

# Evaluation function (Updated)
def evaluate_model(model, val_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    precision, recall, f1, accuracy, balanced_acc, kappa_score, mcc = compute_metrics(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}, Accuracy: {accuracy:.2f}")
    print(f"Balanced Accuracy: {balanced_acc:.2f}, Cohen's Kappa: {kappa_score:.2f}, MCC: {mcc:.2f}")
    print("Confusion Matrix:")
    print(conf_matrix)

    return precision, recall, f1, accuracy, balanced_acc, kappa_score, mcc, conf_matrix
# K-Fold Cross-Validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
fold_results = []
online_adaptation_percentage = 0.3  
# Updated evaluation after training and online adaptation
for fold, (train_index, val_index) in enumerate(kf.split(combined_data)):
    print(f"Fold {fold + 1}/{k}")
    X_train, X_val = combined_data.iloc[train_index].values, combined_data.iloc[val_index].values
    y_train, y_val = labels[train_index], labels[val_index]

    adaptation_size = int(len(X_val) * online_adaptation_percentage)
    buffer_X, buffer_y = X_val[:adaptation_size], y_val[:adaptation_size]
    X_val, y_val = X_val[adaptation_size:], y_val[adaptation_size:]

    train_dataset = EEGEMGDataset(X_train, y_train, noise_std=0.02)
    val_dataset = EEGEMGDataset(X_val, y_val, noise_std=0.0)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model = TCNModel(input_dim=X_train.shape[1], num_classes=len(np.unique(labels)), num_filters=16, dropout_rate=0.7)  # Uncomment for TCN

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

    train_model(model, train_loader, val_loader, criterion, optimizer)
    
    print("\nInitial Evaluation Before Online Learning:")
    evaluate_model(model, val_loader)
    
    online_adaptation(model, optimizer, buffer_X, buffer_y, criterion, val_loader)
    
    print("\nFinal Evaluation After Online Learning:")
    evaluate_model(model, val_loader)


Fold 1/5
Epoch 1/25, Loss: 1.9472, Val Loss: 1.9379
Epoch 2/25, Loss: 1.9454, Val Loss: 1.9351
Epoch 3/25, Loss: 1.9444, Val Loss: 1.9343
Epoch 4/25, Loss: 1.9425, Val Loss: 1.9343
Epoch 5/25, Loss: 1.9400, Val Loss: 1.9313
Epoch 6/25, Loss: 1.9361, Val Loss: 1.9355
Epoch 7/25, Loss: 1.9316, Val Loss: 1.9289
Epoch 8/25, Loss: 1.9252, Val Loss: 1.9223
Epoch 9/25, Loss: 1.9184, Val Loss: 1.9134
Epoch 10/25, Loss: 1.9090, Val Loss: 1.9025
Epoch 11/25, Loss: 1.8985, Val Loss: 1.9120
Epoch 12/25, Loss: 1.8867, Val Loss: 1.8664
Epoch 13/25, Loss: 1.8760, Val Loss: 1.8734
Epoch 14/25, Loss: 1.8598, Val Loss: 1.8696
Epoch 15/25, Loss: 1.8446, Val Loss: 1.8766
Epoch 16/25, Loss: 1.8276, Val Loss: 1.8648
Epoch 17/25, Loss: 1.8116, Val Loss: 1.8346
Epoch 18/25, Loss: 1.7957, Val Loss: 1.8452
Epoch 19/25, Loss: 1.7788, Val Loss: 1.8292
Epoch 20/25, Loss: 1.7611, Val Loss: 1.8360
Epoch 21/25, Loss: 1.7409, Val Loss: 1.8112
Epoch 22/25, Loss: 1.7279, Val Loss: 1.8128
Epoch 23/25, Loss: 1.7049, Val L

/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `

Precision: 0.14, Recall: 0.24, F1 Score: 0.14, Accuracy: 0.24
Balanced Accuracy: 0.20, Cohen's Kappa: 0.07, MCC: 0.10
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   0   1   6   0   0   0]
 [  0   0  40 136   0   0   0]
 [  0   0  11 180   0   0   0]
 [ 15  18   3 149   5   0   0]
 [  0  15  32 104  37   0   0]
 [  0  35  38 102   0   0   0]]
Online Adaptation Cycle 2/3
Validation Metrics After Cycle:
Precision: 0.51, Recall: 0.22, F1 Score: 0.17, Accuracy: 0.22
Balanced Accuracy: 0.30, Cohen's Kappa: 0.15, MCC: 0.20
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   5   0   2   0   0   0]
 [  0  85  12  79   0   0   0]
 [  0   4   0 187   0   0   0]
 [ 57  55   0  76   2   0   0]
 [ 78 110   0   0   0   0   0]
 [  5 170   0   0   0   0   0]]
Online Adaptation Cycle 3/3


/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `

Validation Metrics After Cycle:
Precision: 0.18, Recall: 0.09, F1 Score: 0.12, Accuracy: 0.09
Balanced Accuracy: 0.23, Cohen's Kappa: 0.07, MCC: 0.10
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   7   0   0   0   0   0]
 [  0 176   0   0   0   0   0]
 [  3 110   0  78   0   0   0]
 [100  80   0  10   0   0   0]
 [124  64   0   0   0   0   0]
 [108  67   0   0   0   0   0]]

Final Evaluation After Online Learning:
Precision: 0.18, Recall: 0.09, F1 Score: 0.12, Accuracy: 0.09
Balanced Accuracy: 0.23, Cohen's Kappa: 0.07, MCC: 0.10
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   7   0   0   0   0   0]
 [  0 176   0   0   0   0   0]
 [  3 110   0  78   0   0   0]
 [100  80   0  10   0   0   0]
 [124  64   0   0   0   0   0]
 [108  67   0   0   0   0   0]]
Fold 2/5


...

/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `

Precision: 0.00, Recall: 0.00, F1 Score: 0.00, Accuracy: 0.00
Balanced Accuracy: 0.00, Cohen's Kappa: 0.00, MCC: 0.00
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  2 142   0   0   0   0   0]
 [  0 214   0   0   0   0   0]
 [ 28 150   0   0   0   0   0]
 [ 37 158   0   0   0   0   0]
 [ 47 149   0   0   0   0   0]]
Online Adaptation Cycle 2/3
Validation Metrics After Cycle:
Precision: 0.00, Recall: 0.00, F1 Score: 0.00, Accuracy: 0.00
Balanced Accuracy: 0.00, Cohen's Kappa: 0.00, MCC: 0.00
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0 144   0   0   0   0   0]
 [  0 214   0   0   0   0   0]
 [ 13 165   0   0   0   0   0]
 [ 37 158   0   0   0   0   0]
 [ 47 149   0   0   0   0   0]]
Online Adaptation Cycle 3/3


/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `

Validation Metrics After Cycle:
Precision: 0.00, Recall: 0.00, F1 Score: 0.00, Accuracy: 0.00
Balanced Accuracy: 0.00, Cohen's Kappa: 0.00, MCC: 0.00
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0 144   0   0   0   0   0]
 [  0 214   0   0   0   0   0]
 [  7 171   0   0   0   0   0]
 [ 37 158   0   0   0   0   0]
 [ 42 154   0   0   0   0   0]]

Final Evaluation After Online Learning:
Precision: 0.00, Recall: 0.00, F1 Score: 0.00, Accuracy: 0.00
Balanced Accuracy: 0.00, Cohen's Kappa: 0.00, MCC: 0.00
Confusion Matrix:
[[  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0 144   0   0   0   0   0]
 [  0 214   0   0   0   0   0]
 [  7 171   0   0   0   0   0]
 [ 37 158   0   0   0   0   0]
 [ 42 154   0   0   0   0   0]]


/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tipu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [6]:
import numpy as np

# CNNGRU metrics
cnn_gru_metrics = [
    [0.84, 0.78, 0.76, 0.78, 0.82, 0.72, 0.74],
    [0.79, 0.74, 0.70, 0.74, 0.62, 0.67, 0.70],
    [0.57, 0.55, 0.54, 0.55, 0.54, 0.46, 0.49],
    [0.10, 0.20, 0.12, 0.20, 0.21, 0.05, 0.07],
    [0.39, 0.31, 0.29, 0.31, 0.29, 0.17, 0.19]
]

# Online metrics for CNNGRU
online_metrics_cnn_gru = [
    [0.85, 0.81, 0.81, 0.81, 0.85, 0.77, 0.78],
    [0.61, 0.57, 0.55, 0.57, 0.52, 0.49, 0.53],
    [0.58, 0.50, 0.45, 0.50, 0.47, 0.39, 0.42],
    [0.07, 0.04, 0.05, 0.04, 0.04, 0.02, 0.04],
    [0.34, 0.24, 0.25, 0.24, 0.23, 0.11, 0.12]
]

# TCN metrics
tcn_metrics = [
    [0.33, 0.33, 0.21, 0.33, 0.28, 0.17, 0.21],
    [0.07, 0.22, 0.11, 0.22, 0.18, 0.02, 0.03],
    [0.52, 0.35, 0.29, 0.35, 0.37, 0.21, 0.22],
    [0.15, 0.24, 0.15, 0.24, 0.25, 0.07, 0.09],
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
]

# Online metrics for TCN
online_metrics_tcn = [
    [0.14, 0.24, 0.14, 0.24, 0.20, 0.07, 0.10],
    [0.08, 0.23, 0.12, 0.23, 0.19, 0.04, 0.05],
    [0.35, 0.17, 0.19, 0.17, 0.18, 0.09, 0.11],
    [0.27, 0.18, 0.11, 0.18, 0.19, 0.06, 0.07],
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
]

# Calculate averages
cnn_gru_avg = np.mean(cnn_gru_metrics, axis=0)
online_avg_cnn_gru = np.mean(online_metrics_cnn_gru, axis=0)
tcn_avg = np.mean(tcn_metrics, axis=0)
online_avg_tcn = np.mean(online_metrics_tcn, axis=0)

# Metric labels
metric_labels = [
    "Precision", "Recall", "F1 Score", "Accuracy", "Balanced Accuracy", "Cohen's Kappa", "MCC"
]

# Display results
def display_results(label, averages):
    print(f"Average metrics for {label}:")
    for metric, avg in zip(metric_labels, averages):
        print(f"{metric}: {avg:.2f}")
    print()

#display_results("CNNGRU", cnn_gru_avg)
#display_results("Online CNNGRU", online_avg_cnn_gru)
display_results("TCN", tcn_avg)
display_results("Online TCN", online_avg_tcn)


Average metrics for TCN:
Precision: 0.21
Recall: 0.23
F1 Score: 0.15
Accuracy: 0.23
Balanced Accuracy: 0.22
Cohen's Kappa: 0.09
MCC: 0.11

Average metrics for Online TCN:
Precision: 0.17
Recall: 0.16
F1 Score: 0.11
Accuracy: 0.16
Balanced Accuracy: 0.15
Cohen's Kappa: 0.05
MCC: 0.07

